# 🧠 OOP Part 3: Inheritance, `super()`, and Polymorphism
### From eye color to class hierarchies — reuse and extend behavior
---
In this lesson we build intuition for **inheritance** using a real-life analogy (eye color), then move into Python classes:
- How subclasses inherit data and behavior
- What happens with/without a subclass `__init__` (and what arguments you must pass)
- Why and how to call `super()` (formal definition included)
- Method overriding and extending behavior
- **Polymorphism** (formal definition) — one interface, many behaviors
- Multi-level inheritance (grandparent → parent → child), including calling parent & grandparent
- Hands-on exercises after each concept

## 👁️ 1. Where did you get your eye color?
You likely **inherited** traits from your parents. In Python, classes can do the same: a **child class** (*subclass*) can inherit data (attributes) and behavior (methods) from a **parent class** (*superclass*).

**Vocabulary**
- **Superclass / Parent**: the class being inherited from
- **Subclass / Child**: the class that inherits
- **Inheritance**: the mechanism by which subclasses reuse parent code

We’ll start simple and layer in details gradually.

## 2. Inheriting behavior with no changes
A subclass automatically has its parent’s methods and attributes (unless it overrides them).

In [ ]:
class Parent:
    def speak(self):
        print("I am the parent.")

class Child(Parent):
    def speak(self):
        super().speak()
        print('I am the child')
    def parent_speak(self):
        super().speak()

c = Child()
c.parent_speak()  # ✅ inherited method

### ✏️ Exercise 1 — Your family tree
Create a class `Person` with `__init__(self, name)` and a method `introduce()` that prints the name. Then create `Student(Person)` that doesn’t add anything new. Confirm `Student("Alice").introduce()` works (inherited behavior).

In [ ]:
# TODO: Implement Person and Student (no custom __init__ in Student)

class Person:
    def __init__(self, name):
        self.name = name

    def introduce(self):
        print(self.name)

class Student(Person):
    pass


# Uncomment to test
s = Student("Alice")
s.introduce()

## 3. Constructors and arguments — how `__init__` behaves
When we instantiate a subclass, which constructor runs? It depends:

| Situation | What happens | Rule for creating the object |
|---|---|---|
| Subclass **has no** `__init__` | The parent’s `__init__` is used automatically | You must pass **all arguments required by the parent** |
| Subclass **defines its own** `__init__` | The parent’s `__init__` is **not called automatically** | You must call `super().__init__(...)` to initialize parent state, and your subclass **may require additional args** |

**Key takeaway:**
- If a subclass *doesn’t* define `__init__`, you still need to pass the **parent’s constructor arguments** when creating the child.
- If a subclass *does* define `__init__`, you typically need to pass **both the parent’s required args** (handled via `super().__init__`) **and any new subclass args**.

### 3A. Subclass **without** its own `__init__` (inherits parent constructor)
Because `Student` doesn’t define `__init__`, it inherits `Person.__init__` and therefore expects the **same arguments** as the parent.

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name

class Student(Person):
    pass  # no __init__ -> uses Person.__init__

s = Student("Alice")  # must supply the parent's expected args
print(s.name)  # ✅ 'Alice'

### ✏️ Exercise 2 — Subclass without constructor
Define `Vehicle(make, model)` and subclass `Car(Vehicle)` **without** an `__init__`. Create `Car("Toyota", "Camry")` and print both attributes. Why does this work?

In [ ]:
# TODO: Implement Vehicle and Car (no custom __init__ in Car)

class Vehicle:
    def __init__(self, make, model):
        self.model = model
        self.make = make

class Car(Vehicle):
    pass




# Uncomment to test
c = Car("Toyota", "Camry")
print(c.make, c.model)

### 3B. Subclass **with** its own `__init__` (must call `super()`)
If a subclass defines its own constructor, it *replaces* the parent’s. To preserve parent initialization, you **must** call `super().__init__(...)` with the parent’s required args. You may also accept **additional** arguments specific to the subclass.

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name

class Student(Person):
    def __init__(self, name, student_id):
        super().__init__(name)   # initialize parent part
        self.student_id = student_id  # initialize child part

s = Student("Alice", 1234)
print(s.name, s.student_id)  # ✅ Alice 1234

### ✏️ Exercise 3 — Custom constructor with `super()`
Create `Book(title, author)` and subclass `Textbook(Book)` with an extra `subject`. Use `super().__init__(title, author)` in `Textbook.__init__`. Instantiate and print all three attributes to confirm it works as intended (parent + child args required at creation time).

In [ ]:
# TODO: Implement Book and Textbook using super().__init__

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

class Textbook(Book):
    def __init__(self, title, author, subject):
        super().__init__(title,  author)
        self.subject = subject

# Uncomment to test
t = Textbook("Intro to CS", "Ada", "Programming")
print(t.title, t.author, t.subject)

## 4. Overriding methods — changing behavior in a subclass
A subclass can **override** a method from its parent to provide class-specific behavior.

In [ ]:
class Animal:
    def speak(self):
        print("Some generic sound")

class Dog(Animal):
    def speak(self):  # override
        print("Woof!")

Animal().speak()
Dog().speak()

### ✏️ Exercise 4 — Override behavior
Create `Shape` with `area()` that prints "Area not defined." Then create `Square(Shape)` that overrides `area()` to compute side × side. Instantiate and test both to see different behavior under the same method name.

In [ ]:
# TODO: Implement Shape and Square with overriding
class Shape:
    def area(self):
        print('area not defined')

class Square(Shape):
    def __init__(self, side):
        self.side = side

    def area(self):
        print(self.side ** 2)



# Uncomment to test
Shape().area()
Square(5).area()

## 5. Extending behavior — calling the parent method with `super()`
Sometimes we don’t want to replace the parent’s behavior; we want to **add to it**. Use `super().method(...)` inside the override to call the parent version *and then* add subclass-specific logic.

In [ ]:
class Logger:
    def log(self, message):
        print(f"[LOG] {message}")

class TimestampedLogger(Logger):
    def log(self, message):  # extend behavior
        import datetime
        super().log(f"{datetime.datetime.now()}: {message}")  # parent behavior
        print("[TIM] (Message logged with timestamp)")            # child behavior

TimestampedLogger().log("System started")

### ✏️ Exercise 5 — Combine parent + child behavior
Create `BankAccount.withdraw(amount)` that prints new balance if funds exist, otherwise prints an error. Create `SavingsAccount(BankAccount)` that overrides `withdraw` and **calls `super().withdraw`**, then prints "Interest recalculated" after a successful withdrawal. Test both code paths (success and insufficient funds).

In [ ]:
# TODO: Implement BankAccount and SavingsAccount with super().withdraw


# Uncomment to test
#a = SavingsAccount("Alice", 300)
#a.withdraw(100)
#a.withdraw(500)

## 6. Formal definition — Polymorphism
**Polymorphism** ("many forms") is the ability of different objects to respond to the **same operation** (method call or function) in a way that is **specific to their own class**. A single interface (method name) can have multiple implementations.

**Why it matters**: You can write code that works over a *family* of types without caring about the concrete type at each step.

In [ ]:
class Animal:
    def speak(self):
        print("Some generic sound")

class Dog(Animal):
    def speak(self):
        print("Woof!")

class Cat(Animal):
    def speak(self):
        print("Meow!")

animals = [Animal(), Dog(), Cat()]
for a in animals:
    a.speak()  # same message, different results

### ✏️ Exercise 6 — Polymorphism practice
Create `Rectangle`, `Circle`, and `Triangle`, each with an `area()` method. Store one of each in a list and loop: `for shape in shapes: print(shape.area())`. Do **not** type-check; rely on the shared interface (duck typing).

In [ ]:
# TODO: Implement polymorphic area() methods
import math

class Rectangle:
    pass

class Circle:
    pass
class Triangle:
    pass

# Uncomment to test
# shapes = [Rectangle(3,4), Circle(2), Triangle(3,5)]
# for s in shapes:
#     print(round(s.area(), 2))

#NOTE ABC abstract base case

## 7. Formal definition — `super()` and the MRO
**Formal definition**: `super()` is a **built-in function** that returns a **temporary proxy object** bound to the current instance and class context. It enables calling the **next method in the Method Resolution Order (MRO)** without naming the parent class explicitly. This supports **cooperative inheritance** (especially with multiple inheritance).

- `super()` is not “the parent class”; it’s *the next method in the MRO chain*.
- Prefer `super()` over hard-coding a parent class name to keep code flexible and correct in complex hierarchies.
- View MRO with `ClassName.__mro__`.

In [ ]:
class A: pass
class B(A): pass
class C(B): pass
print(C.__mro__)  # shows the MRO (search order)
print(C.mro())

## 8. Multi-level inheritance — all share & override methods, calling parent & grandparent
When a class hierarchy has more than one level, Python searches for methods following the **Method Resolution Order (MRO)**: `Child → Parent → Grandparent → object`.

Below, all three define **the same method** (`greet`) and **each also has a unique method**. `Parent` also **overrides** `legacy` from `Grandparent`. The `Child` will:
- override `greet`
- call the **parent’s** `greet` (which itself overrides the grandparent)
- call the **grandparent’s** `greet` directly
- call a **parent-only** method and a **grandparent-only** method
- call the **parent’s overridden** `legacy` and then the **grandparent’s original** `legacy`

In [ ]:
class Grandparent:
    def greet(self):
        print("👴 Grandparent says hello!")
    def history(self):
        print("📜 Grandparent shares family history")
    def legacy(self):
        print("💰 Grandparent passes legacy")

class Parent(Grandparent):
    def greet(self):
        print("👨 Parent says hello!")  # overrides grandparent's greet
    def advice(self):
        print("🧭 Parent gives life advice")
    def legacy(self):
        print("🏦 Parent updates the family legacy")  # overrides legacy

class Child(Parent):
    def greet(self):
        print("🧒 Child says hello!")
        # Call parent's greet() (which overrides grandparent)
        super().greet()
        # Call grandparent greet() directly (skip parent)
        Grandparent.greet(self)
    def play(self):
        print("🎮 Child is playing")
    def reflect(self):
        print("💭 Child reflects on family lessons")
        # Call parent-only method
        super().advice()
        # Call parent's overridden legacy (cooperative path)
        super().legacy()
        # Call grandparent's original legacy explicitly
        Grandparent.legacy(self)

c = Child()
c.greet()
print("---")
c.reflect()

### ✏️ Exercise 7 — Three-level hierarchy practice
Build a hierarchy of `Device → Phone → SmartPhone` with this behavior:
1. All define `describe()`, each with a unique message.
2. `Phone` overrides and **enhances** `Device.describe()` (e.g., prints its own message then calls `super().describe()`).
3. `SmartPhone` overrides and:
   - Prints its own message
   - Calls the **parent’s** `describe()` (`super().describe()`)
   - Then calls the **grandparent’s** version **explicitly** (`Device.describe(self)`).
4. Add a unique method at each level (e.g., `power_on()`, `dial()`, `install_app()`) and demonstrate them.

In [ ]:
# TODO: Implement Device, Phone, SmartPhone per the spec above
class Device:
    pass

class Phone(Device):
    pass

class SmartPhone(Phone):
    pass

# Uncomment to test
# s = SmartPhone()
# s.describe()
# s.power_on(); s.dial(); s.install_app("Notes")

## 9. Understanding `type()` vs `isinstance()` — Class Type vs Inheritance Relationships

When a subclass is created, each object made from it is of that subclass’s **type**, but it is also an **instance** of every ancestor in its inheritance chain.

In other words, a `Child` object is recognized as a `Child`, a `Parent`, and a `Grandparent` at the same time — just like a real child is still part of their parent’s and grandparent’s family lines.

### 🧱 Example
```python
class Grandparent:
    pass

class Parent(Grandparent):
    pass

class Child(Parent):
    pass

c = Child()

print(type(c))                     # <class '__main__.Child'>
print(isinstance(c, Child))        # True
print(isinstance(c, Parent))       # True
print(isinstance(c, Grandparent))  # True
```

### 🧠 Explanation
| Function | What it checks | Example |
|-----------|----------------|----------|
| `type(obj)` | Exact class of object | `type(c)` → `Child` |
| `isinstance(obj, Class)` | Whether `obj` is an instance of `Class` **or any of its subclasses** | `isinstance(c, Grandparent)` → True |

### ✅ Key Takeaway
> A subclass instance "is-a" member of every superclass above it.  
That’s the essence of inheritance — shared identity along the hierarchy.

```
Child ⊆ Parent ⊆ Grandparent
```

or in words: *Every Child is a Parent and a Grandparent too.*

## ✅ Summary
| Concept | Core idea | Example |
|---|---|---|
| Inheritance | Reuse parent code in a child class | `class Student(Person)` |
| Constructor rules | No `__init__` → use parent’s and pass parent args; Own `__init__` → call `super().__init__` and accept child args | `Student(name, student_id)` |
| Overriding | Replace parent behavior | `Dog.speak()` |
| Extending via `super()` | Call the parent then add your logic | `SavingsAccount.withdraw()` calls `super()` |
| Polymorphism | Same interface, different behavior | `for a in animals: a.speak()` |
| MRO & `super()` | `super()` finds the next method in the MRO | `Class.__mro__` to inspect order |

> **Mental model:** `super()` isn’t "the parent" — it’s the **next stop** in the method resolution order. If everyone cooperates and calls `super()`, the whole family (parent, grandparent, etc.) gets a turn.